# Add SHN Info to Transit Routes in the middle of the Pipeline

In [1]:
import datetime

import geopandas as gpd
import google.auth
import numpy as np
import pandas as pd
import yaml
from calitp_data_analysis import geography_utils, utils
from calitp_data_analysis.geography_utils import WGS84
from segment_speed_utils import helpers
from shared_utils import (
    catalog_utils,
    portfolio_utils,
    publish_utils,
    rt_dates,
    rt_utils,
    schedule_rt_utils,
)
from update_vars import GTFS_DATA_DICT, RT_SCHED_GCS, SCHED_GCS, SEGMENT_GCS

credentials, project = google.auth.default()

import gcsfs

fs = gcsfs.GCSFileSystem()

In [2]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [3]:
analysis_date_list = rt_dates.y2025_dates

In [4]:
analysis_date_list[0]

'2025-01-15'

In [5]:
date = analysis_date_list[0]

## Load in Routes from `open_data_portal`

In [6]:
trips = helpers.import_scheduled_trips(
    date,
    columns=[
        "gtfs_dataset_key",
        "route_id",
        "route_type",
        "shape_id",
        "shape_array_key",
        "route_long_name",
        "route_short_name",
        "route_desc",
    ],
    get_pandas=True,
).dropna(subset="shape_array_key")

In [7]:
trips.sample()

,schedule_gtfs_dataset_key,route_id,route_type,shape_id,shape_array_key,route_long_name,route_short_name,route_desc
2475,fb467982dcc77a7f9199bebe709bb700,61,3,116030,3edfadf439ae8e6ba958fe466755e532,Sierra & Piedmont - Good Samaritan Hospital,61,None


In [8]:
shapes = helpers.import_scheduled_shapes(
    date, columns=["shape_array_key", "n_trips", "geometry"], get_pandas=True, crs=WGS84
).dropna(subset="shape_array_key")

In [9]:
shapes.sample().drop(columns=["geometry"])

,shape_array_key,n_trips
4240,e2bdccd7b6bcfb652841593e79c390be,3


In [10]:
df = (
    pd.merge(shapes, trips, on="shape_array_key", how="inner")
    .drop_duplicates(subset="shape_array_key")
    .drop(columns="shape_array_key")
)

In [11]:
df.shape

(7416, 9)

In [12]:
drop_cols = ["route_short_name", "route_long_name", "route_desc"]
route_shape_cols = ["schedule_gtfs_dataset_key", "route_id", "shape_id"]

In [13]:
def remove_erroneous_shapes(
    shapes_with_route_info: gpd.GeoDataFrame,
) -> gpd.GeoDataFrame:
    """
    Check if line is simple for Amtrak. If it is, keep.
    If it's not simple (line crosses itself), drop.

    In Jun 2023, some Amtrak shapes appeared to be funky,
    but in prior months, it's been ok.
    Checking for length is fairly time-consuming.
    """
    amtrak = "Amtrak Schedule"

    possible_error = shapes_with_route_info[shapes_with_route_info.name == amtrak]
    ok = shapes_with_route_info[shapes_with_route_info.name != amtrak]

    # Check if the line crosses itself
    ok_amtrak = (
        possible_error.assign(simple=possible_error.geometry.is_simple)
        .query("simple == True")
        .drop(columns="simple")
    )

    ok_shapes = pd.concat([ok, ok_amtrak], axis=0).reset_index(drop=True)

    return ok_shapes

### Didn't reach the step of `routes_assmebled2` because of all the different imports causing issues.

In [14]:
routes_assembled = (
    portfolio_utils.add_route_name(df)
    .drop(columns=drop_cols)
    .sort_values(route_shape_cols)
    .drop_duplicates(subset=route_shape_cols)
    .reset_index(drop=True)
)

In [15]:
routes_assembled.shape

(7416, 7)

In [16]:
routes_assembled.columns

Index(['n_trips', 'geometry', 'schedule_gtfs_dataset_key', 'route_id',
       'route_type', 'shape_id', 'route_name_used'],
      dtype='object')

### Add length to the transit routes.

In [17]:
routes_assembled = routes_assembled.assign(
    route_length_feet=routes_assembled.geometry.to_crs(
        geography_utils.CA_NAD83Albers_ft
    ).length
)

## Load in SHS


In [18]:
def dissolve_shn(columns_to_dissolve: list, file_name: str) -> gpd.GeoDataFrame:
    """
    Dissolve State Highway Network so there will only be one row for each
    route name and route type
    """
    # Read in the dataset and change the CRS to one to feet.
    SHN_FILE = catalog_utils.get_catalog(
        "shared_data_catalog"
    ).state_highway_network.urlpath

    shn = gpd.read_parquet(
        SHN_FILE,
        storage_options={"token": credentials.token},
    ).to_crs(geography_utils.CA_NAD83Albers_ft)

    # Dissolve by route which represents the the route's name and drop the other columns
    # because they are no longer relevant.
    shn_dissolved = (shn.dissolve(by=columns_to_dissolve).reset_index())[
        columns_to_dissolve + ["geometry"]
    ]

    # Rename because I don't want any confusion between SHN route and
    # transit route.
    shn_dissolved = shn_dissolved.rename(columns={"Route": "shn_route"})
    shn_dissolved.columns = shn_dissolved.columns.str.lower()
    # Find the length of each highway.
    shn_dissolved = shn_dissolved.assign(
        highway_feet=shn_dissolved.geometry.length,
        shn_route=shn_dissolved.shn_route.astype(int).astype(str),
    )

    # Save this out so I don't have to dissolve it each time.
    shn_dissolved.to_parquet(
        f"gs://calitp-analytics-data/data-analyses/state_highway_network/shn_dissolved_by_{file_name}.parquet",
        filesystem=fs,
    )
    return shn_dissolved

In [19]:
# dissolved_route = dissolve_shn(["Route", "District"], "ct_district_route")

In [20]:
dissolved_url = "gs://calitp-analytics-data/data-analyses/state_highway_network/shn_dissolved_by_ct_district_route.parquet"

In [21]:
dissolved_df = gpd.read_parquet(
    dissolved_url,
    storage_options={"token": credentials.token},
)

In [22]:
def buffer_shn(buffer_amount: int, file_name: str) -> gpd.GeoDataFrame:
    """
    Add a buffer to the SHN before overlaying it with
    transit routes.
    """
    GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/state_highway_network/"

    # Read in the dissolved SHN file
    shn_df = gpd.read_parquet(
        f"{GCS_FILE_PATH}shn_dissolved_by_{file_name}.parquet",
        storage_options={"token": credentials.token},
    )

    # Buffer the state highway.
    shn_df_buffered = shn_df.assign(
        geometry=shn_df.geometry.buffer(buffer_amount),
    )

    # Save it out so we won't have to buffer over again and
    # can just read it in.
    shn_df_buffered.to_parquet(
        f"{GCS_FILE_PATH}shn_buffered_{buffer_amount}_ft_{file_name}.parquet",
        filesystem=fs,
    )

    return shn_df_buffered

In [23]:
SHN_HWY_BUFFER_FEET = 50
PARALLEL_HWY_BUFFER_FEET = geography_utils.FEET_PER_MI * 0.5

In [24]:
# buffered_df = buffer_shn(SHN_HWY_BUFFER_FEET, "ct_district_route")

In [25]:
shn_district_df = gpd.read_parquet(
    f"gs://calitp-analytics-data/data-analyses/state_highway_network/shn_buffered_50_ft_ct_district_route.parquet",
    storage_options={"token": credentials.token},
)

In [26]:
len(shn_district_df)

344

In [27]:
shn_district_df.columns

Index(['shn_route', 'district', 'geometry', 'highway_feet'], dtype='object')

## Overlay the transit routes with the SHN 

In [28]:
def routes_shn_intersection(
    routes_gdf: gpd.GeoDataFrame, buffer_amount: int, file_name: str
) -> gpd.GeoDataFrame:
    """
    Overlay the most recent transit routes with a buffered version
    of the SHN
    """
    GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/state_highway_network/"

    # Read in buffered shn here or re buffer if we don't have it available.
    HWY_FILE = f"{GCS_FILE_PATH}shn_buffered_{buffer_amount}_ft_{file_name}.parquet"

    if fs.exists(HWY_FILE):
        shn_routes_gdf = gpd.read_parquet(
            HWY_FILE, storage_options={"token": credentials.token}
        )
    else:
        shn_routes_gdf = buffer_shn(buffer_amount)

    # Process the most recent transit route geographies and ensure the
    # CRS matches the SHN routes' GDF so the overlay doesn't go wonky.
    routes_gdf = routes_gdf.to_crs(shn_routes_gdf.crs)

    # Overlay transit routes with the SHN geographies.
    gdf = gpd.overlay(
        routes_gdf, shn_routes_gdf, how="intersection", keep_geom_type=True
    )

    # Calcuate the percent of the transit route that runs on a highway, round it up and
    # multiply it by 100. Drop the geometry because we want the original transit route
    # shapes.
    gdf = gdf.assign(
        pct_route_on_hwy=(gdf.geometry.length / gdf.route_length_feet).round(3) * 100,
    ).drop(
        columns=[
            "geometry",
        ]
    )

    # Join back the dataframe above with the original transit route dataframes
    # so we can have the original transit route geographies.
    gdf2 = pd.merge(
        routes_gdf,
        gdf,
        on=[
            "n_trips",
            "schedule_gtfs_dataset_key",
            "route_id",
            "route_type",
            "shape_id",
            "route_name_used",
            "route_length_feet",
        ],
        how="left",
    )

    # Clean up
    gdf2.district = gdf2.district.fillna(0).astype(int)
    return gdf2

In [29]:
routes_assembled.columns

Index(['n_trips', 'geometry', 'schedule_gtfs_dataset_key', 'route_id',
       'route_type', 'shape_id', 'route_name_used', 'route_length_feet'],
      dtype='object')

In [30]:
intersecting = routes_shn_intersection(routes_assembled, 50, "ct_district_route")

In [31]:
len(intersecting)

20476

In [32]:
intersecting.pct_route_on_hwy.describe()

count   18951.00
mean        6.65
std        15.80
min         0.00
25%         0.10
50%         0.40
75%         2.70
max        99.20
Name: pct_route_on_hwy, dtype: float64

In [33]:
intersecting.columns

Index(['n_trips', 'geometry', 'schedule_gtfs_dataset_key', 'route_id',
       'route_type', 'shape_id', 'route_name_used', 'route_length_feet',
       'shn_route', 'district', 'highway_feet', 'pct_route_on_hwy'],
      dtype='object')

### Find multi route districts

In [34]:
# Find routes that cross multiple districts
multi_district_routes = (
    intersecting.groupby(["schedule_gtfs_dataset_key", "route_name_used", "route_id"])
    .agg({"district": "nunique"})
    .reset_index()
)

In [35]:
multi_district_routes.district.describe()

count   2683.00
mean       1.09
std        0.38
min        1.00
25%        1.00
50%        1.00
75%        1.00
max        7.00
Name: district, dtype: float64

In [36]:
multi_district_routes.sort_values(by=["district"], ascending=False).head(30)

,schedule_gtfs_dataset_key,route_name_used,route_id,district
1620,a37760dde6b9fdcb76b82e57afab7274,Greyhound US0831,US0831,7
1513,a37760dde6b9fdcb76b82e57afab7274,FlixBus N2003,N2003,6
555,48e137bc977da88970393f629c18432c,Coast Starlight,36924,5
549,48e137bc977da88970393f629c18432c,California Zephyr,96,4
1615,a37760dde6b9fdcb76b82e57afab7274,Greyhound US0800,US0800,4
1616,a37760dde6b9fdcb76b82e57afab7274,Greyhound US0802,US0802,4
1617,a37760dde6b9fdcb76b82e57afab7274,Greyhound US0810,US0810,4
582,48e137bc977da88970393f629c18432c,Texas Eagle,87,4
1619,a37760dde6b9fdcb76b82e57afab7274,Greyhound US0830,US0830,4
1618,a37760dde6b9fdcb76b82e57afab7274,Greyhound US0811,US0811,4


## Routes that overlap with multiple SHN now have 1+ row. Change it so one route will only have one row.

In [37]:
intersecting.columns

Index(['n_trips', 'geometry', 'schedule_gtfs_dataset_key', 'route_id',
       'route_type', 'shape_id', 'route_name_used', 'route_length_feet',
       'shn_route', 'district', 'highway_feet', 'pct_route_on_hwy'],
      dtype='object')

In [38]:
def group_route_district(df: pd.DataFrame, pct_route_on_hwy_agg: str) -> pd.DataFrame:
    """
    Aggregate by adding all the districts and SHN to a single row, rather than
    multiple and sum up the total % of SHN a transit route intersects with.

    df: the dataframe you want to aggregate
    pct_route_on_hwy_agg: whether you want to find the max, min, sum, etc on the column
    "pct_route_on_hwy_across_districts"
    """

    agg1 = (
        df.groupby(
            ["schedule_gtfs_dataset_key", "route_type", "shape_id", "route_id", "route_name_used"],
            as_index=False,
        )[["shn_route", "district", "pct_route_on_hwy_across_districts"]]
        .agg(
            {
                "shn_route": lambda x: ", ".join(set(x.astype(str))),
                "district": lambda x: ", ".join(set(x.astype(str))),
                "pct_route_on_hwy_across_districts": pct_route_on_hwy_agg,
            }
        )
        .reset_index(drop=True)
    )

    # Clean up
    agg1.pct_route_on_hwy_across_districts = (
        agg1.pct_route_on_hwy_across_districts.astype(float).round(2)
    )
    return agg1

In [39]:
def create_on_shs_column(df):
    df["on_shs"] = np.where(df["pct_route_on_hwy_across_districts"] == 0, "N", "Y")
    return df

In [40]:
def add_shn_information(gdf: gpd.GeoDataFrame, buffer_amt:int) -> pd.DataFrame:
    """
    Prepare the gdf to join with the existing transit_routes
    dataframe that is published on the Open Data Portal
    """
    # Overlay
    intersecting = routes_shn_intersection(gdf, 50, "ct_district_route")
    # Rename column
    gdf = gdf.rename(columns={"pct_route_on_hwy": "pct_route_on_hwy_across_districts"})
    # Group the dataframe so that one route only has one
    # row instead of multiple rows after finding its
    # intersection with any SHN routes.
    agg1 = group_route_district(gdf, "sum")

    # Add yes/no column to signify if a transit route intersects
    # with a SHN route
    agg1 = create_on_shs_column(agg1)

    # Clean up rows that are tagged as "on_shs==N" but still have values
    # that appear. 
    agg1.loc[(agg1['on_shs'] == "N") & (agg1['district'] != "0"), 
                        ['district', 'shn_route']] = np.nan
    return agg1

In [41]:
open_data_portal_df = add_shn_information(intersecting, SHN_HWY_BUFFER_FEET)

In [42]:
len(open_data_portal_df)

7416

In [51]:
open_data_portal_df.columns

Index(['schedule_gtfs_dataset_key', 'route_type', 'shape_id', 'route_id',
       'route_name_used', 'shn_route', 'district',
       'pct_route_on_hwy_across_districts', 'on_shs'],
      dtype='object')

In [43]:
open_data_portal_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7416 entries, 0 to 7415
Data columns (total 9 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   schedule_gtfs_dataset_key          7416 non-null   object 
 1   route_type                         7416 non-null   object 
 2   shape_id                           7416 non-null   object 
 3   route_id                           7416 non-null   object 
 4   route_name_used                    7416 non-null   object 
 5   shn_route                          7407 non-null   object 
 6   district                           7407 non-null   object 
 7   pct_route_on_hwy_across_districts  7416 non-null   float64
 8   on_shs                             7416 non-null   object 
dtypes: float64(1), object(8)
memory usage: 521.6+ KB


In [44]:
open_data_portal_df.pct_route_on_hwy_across_districts.describe()

count   7416.00
mean      16.98
std       27.33
min        0.00
25%        0.40
50%        1.40
75%       21.42
max      100.00
Name: pct_route_on_hwy_across_districts, dtype: float64

In [45]:
open_data_portal_df.on_shs.value_counts()

Y    5882
N    1534
Name: on_shs, dtype: int64

In [46]:
open_data_portal_df.columns

Index(['schedule_gtfs_dataset_key', 'route_type', 'shape_id', 'route_id',
       'route_name_used', 'shn_route', 'district',
       'pct_route_on_hwy_across_districts', 'on_shs'],
      dtype='object')

## How is possible to have `on_shs==N` but there are populated values in `shn_route` and `District`

In [47]:
open_data_portal_df.loc[open_data_portal_df.route_name_used == "Southwest Chief"]

,schedule_gtfs_dataset_key,route_type,shape_id,route_id,route_name_used,shn_route,district,pct_route_on_hwy_across_districts,on_shs
1739,48e137bc977da88970393f629c18432c,2,118,51,Southwest Chief,NaN,NaN,0.00,N
1794,48e137bc977da88970393f629c18432c,2,270,51,Southwest Chief,NaN,NaN,0.00,N


In [49]:
open_data_portal_df.loc[(open_data_portal_df.on_shs == "N") & (open_data_portal_df.district != "0")]

,schedule_gtfs_dataset_key,route_type,shape_id,route_id,route_name_used,shn_route,district,pct_route_on_hwy_across_districts,on_shs
1739,48e137bc977da88970393f629c18432c,2,118,51,Southwest Chief,NaN,NaN,0.00,N
1755,48e137bc977da88970393f629c18432c,2,173,96,California Zephyr,NaN,NaN,0.00,N
1756,48e137bc977da88970393f629c18432c,2,174,96,California Zephyr,NaN,NaN,0.00,N
1766,48e137bc977da88970393f629c18432c,2,195,87,Texas Eagle,NaN,NaN,0.00,N
1772,48e137bc977da88970393f629c18432c,2,222,26025,San Joaquins,NaN,NaN,0.00,N
1774,48e137bc977da88970393f629c18432c,2,224,26025,San Joaquins,NaN,NaN,0.00,N
1792,48e137bc977da88970393f629c18432c,2,269,36930,Sunset Limited,NaN,NaN,0.00,N
1794,48e137bc977da88970393f629c18432c,2,270,51,Southwest Chief,NaN,NaN,0.00,N
1795,48e137bc977da88970393f629c18432c,2,271,36930,Sunset Limited,NaN,NaN,0.00,N


In [50]:
open_data_portal_df.columns

Index(['schedule_gtfs_dataset_key', 'route_type', 'shape_id', 'route_id',
       'route_name_used', 'shn_route', 'district',
       'pct_route_on_hwy_across_districts', 'on_shs'],
      dtype='object')

### Map

In [ ]:
m = shn_district_df.explore(
    name="district",
    tiles="CartoDB positron",
    style_kwds={"color": "#9DA4A6", "opacity": 0.5},
    height=500,
    width=1000,
    legend = False
)

In [ ]:
southwest_chief = intersecting.loc[ (intersecting.route_name_used ==  "Southwest Chief") 
]

In [ ]:
"""southwest_chief.explore(
    m=m,
    cmap="Spectral",
    categorical=True,
    legend=False,
    legend_kwds={"width": 200},
)"""